# Introduction to the Reverse Osmosis Unit Model
This tutorial shows how to build, initialize, simulate, and optimize a reverse osmosis (RO) unit model using WaterTAP. In this example, we will minimize the specific energy consumption of the RO unit.

## Step 1: Import libraries.

In [1]:
# Imports from Pyomo, including "value" for getting the 
# value of Pyomo objects
from pyomo.environ import ConcreteModel, Objective, Expression, value

# Imports from IDAES
# Import flowsheet block from IDAES core
from idaes.core import FlowsheetBlock
# Import function to get default solver
from idaes.core.solvers import get_solver
# Import function to check degrees of freedom
from idaes.core.util.model_statistics import degrees_of_freedom
# Import utility function for calculating scaling factors
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor

# Imports from WaterTAP
# Import NaCl property model
from watertap.property_models.NaCl_prop_pack import NaClParameterBlock
# Import RO model
from watertap.unit_models.reverse_osmosis_0D import (ReverseOsmosis0D,
        ConcentrationPolarizationType, MassTransferCoefficient)

## Step 2: Start building the RO model.

In [2]:
# Create a Pyomo concrete model, flowsheet, and NaCl property parameter block.
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = NaClParameterBlock()

In [3]:
# Add an RO unit to the flowsheet.
m.fs.unit = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type=ConcentrationPolarizationType.none,
    mass_transfer_coefficient=MassTransferCoefficient.none,
    has_pressure_change=False,
    )

## Step 3: Specify values for system variables.

In [4]:
m.fs.unit.inlet.flow_mass_phase_comp[0, 'Liq', 'NaCl'].fix(0.035)  # mass flow rate of NaCl (kg/s)
m.fs.unit.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(0.965)   # mass flow rate of water (kg/s)
m.fs.unit.inlet.pressure[0].fix(50e5)                              # feed pressure (Pa)
m.fs.unit.inlet.temperature[0].fix(298.15)                         # feed temperature (K)
m.fs.unit.area.fix(50)                                             # membrane area (m^2)
m.fs.unit.A_comp.fix(4.2e-12)                                      # membrane water permeability (m/Pa/s)
m.fs.unit.B_comp.fix(3.5e-8)                                       # membrane salt permeability (m/s)
m.fs.unit.permeate.pressure[0].fix(101325)                         # permeate pressure (Pa)

## Step 4: Scale all variables.

In [5]:
# Set scaling factors for component mass flowrates.
m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1, index=('Liq', 'H2O'))
m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e2, index=('Liq', 'NaCl'))

# Set scaling factor for membrane area.
set_scaling_factor(m.fs.unit.area, 1e-2)

# Calculate scaling factors for all other variables.
calculate_scaling_factors(m)

## Step 5: Initialize the model.

In [6]:
m.fs.unit.initialize()

2023-10-10 15:47:50 [INFO] idaes.init.fs.unit.feed_side: Initialization Complete
2023-10-10 15:47:50 [INFO] idaes.init.fs.unit: Initialization Complete: optimal - Optimal Solution Found


## Step 6: Setup a solver and run a simulation.

In [7]:
# Check that degrees of freedom = 0 before attempting simulation.
# This means that the performance of the flowsheet is completely
# determined by the system variables that were fixed above.
assert degrees_of_freedom(m) == 0

In [8]:
# Setup solver
solver = get_solver()

In [9]:
# Run simulation
simulation_results = solver.solve(m)

In [10]:
# Display report, reports include a small subset of the most important variables
m.fs.unit.report()


Unit : fs.unit                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                        : Value   : Units         : Fixed : Bounds
                 Membrane Area :  50.000 :    meter ** 2 :  True : (0.1, 100000.0)
    Solvent Mass Recovery Rate : 0.30933 : dimensionless : False : (0.01, 0.999999)
      Volumetric Recovery Rate : 0.30646 : dimensionless : False : (0.01, 0.999999)

------------------------------------------------------------------------------------
    Stream Table
                                               Units         Feed Inlet  Feed Outlet  Permeate Outlet
    flow_mass_phase_comp ('Liq', 'H2O')   kilogram / second     0.96500     0.66649        0.29851   
    flow_mass_phase_comp ('Liq', 'NaCl')  kilogram / second    0.035000    0.034924     7.5676e-05   
    temperature                                      kel

In [11]:
# Display all results, this shows all variables and constraints
m.fs.unit.display()

Block fs.unit

  Variables:
    area : Total Membrane area
        Size=1, Index=None, Units=m**2
        Key  : Lower : Value : Upper    : Fixed : Stale : Domain
        None :   0.1 :    50 : 100000.0 :  True :  True : NonNegativeReals
    _flow_mass_phase_comp_inlet_ref : Size=2, Index=fs.unit._flow_mass_phase_comp_inlet_ref_index, ReferenceTo=fs.unit.feed_side.properties_in[...].component('flow_mass_phase_comp')[...]
        Key                  : Lower : Value : Upper : Fixed : Stale : Domain
         (0.0, 'Liq', 'H2O') :   0.0 : 0.965 :  None :  True :  True : NonNegativeReals
        (0.0, 'Liq', 'NaCl') :   0.0 : 0.035 :  None :  True :  True : NonNegativeReals
    _temperature_inlet_ref : Size=1, Index=fs._time, ReferenceTo=fs.unit.feed_side.properties_in[...].component('temperature')
        Key : Lower  : Value  : Upper  : Fixed : Stale : Domain
        0.0 : 273.15 : 298.15 : 373.15 :  True :  True : NonNegativeReals
    _pressure_inlet_ref : Size=1, Index=fs._time, Refere

## Step 7: Unfix variables, set variable bounds, and run optimization to minimize specific energy consumption.

In [12]:
# Unfix membrane area and feed pressure
m.fs.unit.area.unfix()                  # membrane area (m^2)
m.fs.unit.inlet.pressure[0].unfix()     # feed pressure (Pa)

In [13]:
# Set lower and upper bounds for membrane area (m^2)
m.fs.unit.area.setlb(1)
m.fs.unit.area.setub(500)

In [14]:
# Set lower and upper bounds for feed pressure (Pa)
m.fs.unit.inlet.pressure[0].setlb(10e5)
m.fs.unit.inlet.pressure[0].setub(80e5)

In [15]:
# Assume 100% efficiency of pumps and ERD and no pressure losses
#--> Pump power consumption ~ Qp*Pf/3.6e6
m.fs.specific_energy_consumption = Expression(
    expr=m.fs.unit.inlet.pressure[0]/(3.6e6))

In [16]:
# Define objective function to minimize the specific energy consumption.
m.fs.objective = Objective(expr=m.fs.specific_energy_consumption)

In [17]:
# Set the water recovery to 50%
m.fs.unit.recovery_vol_phase[0,'Liq'].fix(0.50)

In [18]:
# The solver will find the membrane area and 
# inlet pressure that achieve 50% recovery while minimizing
# specific energy consumption. Since we fixed the 
# volumetric water recovery, a degree of freedom 
# was removed from the model and is now 1.
print(degrees_of_freedom(m))

1


In [19]:
# Solve the model
optimization_results = solver.solve(m)
print(optimization_results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 92
  Number of variables: 93
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.0733633041381836
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [20]:
# membrane area of the optimized RO unit
value(m.fs.unit.area)

143.93509265169624

In [21]:
# inlet pressure of the optimized RO unit
value(m.fs.unit.inlet.pressure[0])

4526950.047231534

In [22]:
# the minimum specific energy consumption
value(m.fs.specific_energy_consumption)

1.2574861242309814

In [23]:
# display the overall report on the RO unit
m.fs.unit.report()


Unit : fs.unit                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                        : Value   : Units         : Fixed : Bounds
                 Membrane Area :  143.94 :    meter ** 2 : False : (1, 500)
    Solvent Mass Recovery Rate : 0.50466 : dimensionless : False : (0.01, 0.999999)
      Volumetric Recovery Rate : 0.50000 : dimensionless :  True : (0.01, 0.999999)

------------------------------------------------------------------------------------
    Stream Table
                                               Units         Feed Inlet  Feed Outlet  Permeate Outlet
    flow_mass_phase_comp ('Liq', 'H2O')   kilogram / second     0.96500     0.47801        0.48699   
    flow_mass_phase_comp ('Liq', 'NaCl')  kilogram / second    0.035000    0.034778     0.00022231   
    temperature                                      kelvin    